In [1]:
# !pip install torchsummary
# !pip install torchgeometry
# !pip install git+https://github.com/mberkay0/pretrained-backbones-unet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.3 MB/s eta 0:00:00
  Cloning https://github.com/mberkay0/pretrained-backbones-unet to /tmp/pip-req-build-mporgmld
  Running command git clone --filter=blob:none --quiet https://github.com/mberkay0/pretrained-backbones-unet /tmp/pip-req-build-mporgmld
  Resolved https://github.com/mberkay0/pretrained-backbones-unet to commit c2e9bea6caef0883f1f34d40ecaebb8c374e39e0
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 774.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from torchsummary import summary
from torchgeometry.losses import one_hot
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import time
import imageio
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import Tensor
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torchvision.transforms import *
from collections import OrderedDict
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from backbones_unet.model.unet import Unet

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8bbcafeb-a7e7-1fb4-7fac-c735c1928772)
GPU 1: Tesla T4 (UUID: GPU-246b9022-5dd1-84e7-4164-5818c91a36f0)


In [4]:
torch.cuda.empty_cache()

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Parameters

In [6]:
# Number of class in the data set (3: neoplastic, non neoplastic, background)
num_classes = 3

# Number of epoch
epochs = 50

# Hyperparameters for training 
learning_rate = 2e-04
batch_size = 4
display_step = 50

# Model path
checkpoint_path = '/kaggle/working/unet_model.pth'
pretrained_path = "/kaggle/input/unet-checkpoint/unet_model.pth"
# Initialize lists to keep track of loss and accuracy
loss_epoch_array = []
train_accuracy = []
test_accuracy = []
valid_accuracy = []

# Dataloader

In [7]:
transform = Compose([Resize((512, 512), interpolation=InterpolationMode.BILINEAR),
                     PILToTensor()])

augmentation_1 = Compose([
                        Resize((512, 512), interpolation=InterpolationMode.BILINEAR),
                        RandomHorizontalFlip(p=1),
                        PILToTensor()
                        ])

augmentation_2 = Compose([
                        Resize((512, 512), interpolation=InterpolationMode.BILINEAR),
                        RandomVerticalFlip(p=1),
                        PILToTensor()
                        ])

augmentation_3 = Compose([
                        Resize((512, 512), interpolation=InterpolationMode.BILINEAR),
                        RandomVerticalFlip(p=1),
                        RandomHorizontalFlip(p=1),
                        PILToTensor()
                        ])


In [8]:
class UNetDataClass(Dataset):
    def __init__(self, images_path, masks_path, transform=None):
        super(UNetDataClass, self).__init__()
        
        images_list = os.listdir(images_path)
        masks_list = os.listdir(masks_path)
        
        images_list = [images_path + image_name for image_name in images_list]
        masks_list = [masks_path + mask_name for mask_name in masks_list]
        
        self.images_list = images_list
        self.masks_list = masks_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.images_list[index]
        mask_path = self.masks_list[index]
        
        # Open image and mask
        data = Image.open(img_path)
        label = Image.open(mask_path)
            
        if (self.transform):
            data = self.transform(data) / 255
            label = self.transform(label) / 255

        
        label = torch.where(label>0.65, 1.0, 0.0)
        
        label[2, :, :] = 0.0001
        label = torch.argmax(label, 0).type(torch.int64)
        
        return data, label
    
    def __len__(self):
        return len(self.images_list)



In [9]:
images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
masks_path =  "/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/"

In [10]:
unet_dataset_not_aug = UNetDataClass(images_path, masks_path, transform=transform)
unet_dataset_aug_1 = UNetDataClass(images_path, masks_path, transform=augmentation_1)
unet_dataset_aug_2 = UNetDataClass(images_path, masks_path, transform=augmentation_2)
unet_dataset_aug_3 = UNetDataClass(images_path, masks_path, transform=augmentation_3)
unet_dataset_1 = ConcatDataset([unet_dataset_not_aug, unet_dataset_aug_1])
unet_dataset_2 = ConcatDataset([unet_dataset_1, unet_dataset_aug_2])
unet_dataset = ConcatDataset([unet_dataset_2, unet_dataset_aug_3])
# unet_dataset = unet_dataset_not_aug
print(len(unet_dataset))

4000


In [11]:
train_size = 0.8
valid_size = 0.2

In [12]:
train_set, valid_set = random_split(unet_dataset, 
                                    [int(train_size * len(unet_dataset)) , 
                                     int(valid_size * len(unet_dataset))])

In [13]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

# Loss function

In [18]:
class CEDiceLoss(nn.Module):
    def __init__(self, weights) -> None:
        super(CEDiceLoss, self).__init__()
        self.eps: float = 1e-6
        self.weights: torch.Tensor = weights

    def forward(
            self,
            input: torch.Tensor,
            target: torch.Tensor) -> torch.Tensor:
        if not torch.is_tensor(input):
            raise TypeError("Input type is not a torch.Tensor. Got {}"
                            .format(type(input)))
        if not len(input.shape) == 4:
            raise ValueError("Invalid input shape, we expect BxNxHxW. Got: {}"
                             .format(input.shape))
        if not input.shape[-2:] == target.shape[-2:]:
            raise ValueError("input and target shapes must be the same. Got: {}"
                             .format(input.shape, input.shape))
        if not input.device == target.device:
            raise ValueError(
                "input and target must be in the same device. Got: {}" .format(
                    input.device, target.device))
        if not self.weights.shape[1] == input.shape[1]:
            raise ValueError("The number of weights must equal the number of classes")
        if not torch.sum(self.weights).item() == 1:
            raise ValueError("The sum of all weights must equal 1")
            
        # cross entropy loss
        celoss = nn.CrossEntropyLoss(self.weights)(input, target)
        
        # compute softmax over the classes axis
        input_soft = F.softmax(input, dim=1)

        # create the labels one hot tensor
        target_one_hot = one_hot(target, num_classes=input.shape[1],
                                 device=input.device, dtype=input.dtype)

        # compute the actual dice score
        dims = (2, 3)
        intersection = torch.sum(input_soft * target_one_hot, dims)
        cardinality = torch.sum(input_soft + target_one_hot, dims)

        dice_score = 2. * intersection / (cardinality + self.eps)
        
        dice_score = torch.sum(dice_score * self.weights, dim=1)
        
        return torch.mean(1. - dice_score) + celoss

# Training

**Initialize weights**

In [19]:
def weights_init(model):
    if isinstance(model, nn.Linear):
        # Xavier Distribution
        torch.nn.init.xavier_uniform_(model.weight)

In [20]:
def save_model(model, optimizer, path):
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, path)

def load_model(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer

**Train model**

In [21]:
# Train function for each epoch
def train(train_dataloader, valid_dataloader,learing_rate_scheduler, epoch, display_step):
    print(f"Start epoch #{epoch+1}, learning rate for this epoch: {learing_rate_scheduler.get_last_lr()}")
    start_time = time.time()
    train_loss_epoch = 0
    test_loss_epoch = 0
    last_loss = 999999999
    model.train()
    for i, (data,targets) in enumerate(train_dataloader):
        
        # Load data into GPU
        data, targets = data.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(data)

        # Backpropagation, compute gradients
        loss = loss_function(outputs, targets.long())
        loss.backward()

        # Apply gradients
        optimizer.step()
        
        # Save loss
        train_loss_epoch += loss.item()
        if (i+1) % display_step == 0:
#             accuracy = float(test(test_loader))
            print('Train Epoch: {} [{}/{} ({}%)]\tLoss: {:.4f}'.format(
                epoch + 1, (i+1) * len(data), len(train_dataloader.dataset), 100 * (i+1) * len(data) / len(train_dataloader.dataset), 
                loss.item()))
                  
    print(f"Done epoch #{epoch+1}, time for this epoch: {time.time()-start_time}s")
    train_loss_epoch/= (i + 1)
    
    # Evaluate the validation set
    model.eval()
    with torch.no_grad():
        for data, target in valid_dataloader:
            data, target = data.to(device), target.to(device)
            test_output = model(data)
            test_loss = loss_function(test_output, target)
            test_loss_epoch += test_loss.item()
            
    test_loss_epoch/= (i+1)
    
    return train_loss_epoch , test_loss_epoch

In [23]:
model = Unet(
    in_channels=3,            
    num_classes=num_classes,      
)
model.apply(weights_init)
model = nn.DataParallel(model)
model.to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


DataParallel(
  (module): Unet(
    (encoder): FeatureListNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act2): ReLU(inplace=True)
          (aa): Identity()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
weights = torch.Tensor([[0.4, 0.55, 0.05]]).cuda()
loss_function = CEDiceLoss(weights)

# Define the optimizer (Adam optimizer)
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)
# optimizer.load_state_dict(checkpoint['optimizer'])

# Learning rate scheduler
learing_rate_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.6)

In [25]:
save_model(model, optimizer, checkpoint_path)

In [26]:
wandb.login(
    key = "a561d9f8d588433211a8762b70588097e48c01df",
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
wandb.init(
    project = "assignment_3"
)

# Training loop
train_loss_array = []
test_loss_array = []
last_loss = 9999999999999
for epoch in range(epochs):
    train_loss_epoch = 0
    test_loss_epoch = 0
    (train_loss_epoch, test_loss_epoch) = train(train_loader, 
                                              valid_loader, 
                                              learing_rate_scheduler, epoch, display_step)
    
    if test_loss_epoch < last_loss:
        save_model(model, optimizer, checkpoint_path)
        last_loss = test_loss_epoch
        
    learing_rate_scheduler.step()
    train_loss_array.append(train_loss_epoch)
    test_loss_array.append(test_loss_epoch)
    wandb.log({"Train loss": train_loss_epoch, "Valid loss": test_loss_epoch})


wandb: Currently logged in as: binh-nt2003bg (not_having_team_yet). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.12
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231116_022306-87s05h66
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run jolly-music-57
wandb: ⭐️ View project at https://wandb.ai/not_having_team_yet/assignment_3
wandb: 🚀 View run at https://wandb.ai/not_having_team_yet/assignment_3/runs/87s05h66


Start epoch #1, learning rate for this epoch: [0.0002]
Train Epoch: 1 [200/3200 (6.25%)]	Loss: 1.8361
Train Epoch: 1 [400/3200 (12.5%)]	Loss: 1.4634
Train Epoch: 1 [600/3200 (18.75%)]	Loss: 1.3930
Train Epoch: 1 [800/3200 (25.0%)]	Loss: 1.2424
Train Epoch: 1 [1000/3200 (31.25%)]	Loss: 1.3889
Train Epoch: 1 [1200/3200 (37.5%)]	Loss: 1.1769
Train Epoch: 1 [1400/3200 (43.75%)]	Loss: 1.2127
Train Epoch: 1 [1600/3200 (50.0%)]	Loss: 1.3545
Train Epoch: 1 [1800/3200 (56.25%)]	Loss: 1.2727
Train Epoch: 1 [2000/3200 (62.5%)]	Loss: 1.1528
Train Epoch: 1 [2200/3200 (68.75%)]	Loss: 1.1318
Train Epoch: 1 [2400/3200 (75.0%)]	Loss: 1.2358
Train Epoch: 1 [2600/3200 (81.25%)]	Loss: 1.0296
Train Epoch: 1 [2800/3200 (87.5%)]	Loss: 1.3660
Train Epoch: 1 [3000/3200 (93.75%)]	Loss: 0.9280
Train Epoch: 1 [3200/3200 (100.0%)]	Loss: 0.8399
Done epoch #1, time for this epoch: 333.3135452270508s
Start epoch #2, learning rate for this epoch: [0.0002]
Train Epoch: 2 [200/3200 (6.25%)]	Loss: 0.8890
Train Epoch: 2 [